In [ ]:
# Download a file containing exchange rates
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv

--2024-09-25 16:28:43--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45 [text/csv]
Saving to: ‘exchange_rate.csv’

exchange_rate.csv   100%[===================>]      45  --.-KB/s    in 0s      

2024-09-25 16:28:44 (16.9 MB/s) - ‘exchange_rate.csv’ saved [45/45]



In [ ]:
# Code for ETL operations on Country-GDP data
# Importing the required libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import numpy as np
from datetime import datetime

def log_progress(message):
    ''' This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing'''
    with open('code_log.txt', 'a') as log_file:
        log_file.write(f"{datetime.now()}: {message}\\n")

# Define the known variables
url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
table_attribs = ['Name', 'MC_USD_Billion']
output_path = './Largest_banks_data.csv'
db_name = 'Banks.db'
table_name = 'Largest_banks'
csv_path = 'exchange_rate.csv'

# Log the initial message
log_progress("Preliminaries complete. Initiating ETL process")

In [ ]:
#Define the extract function
def extract(url, table_attribs):
    '''This function aims to extract the required information from the website and save it to a data frame. The function returns the data frame for further processing.'''
    #Send a GET request to the URL
    response = requests.get(url)
    #Parse the response content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    #Locate the first table on the webpage
    table = soup.find('table')
    #Get all the rows from the table
    rows = table.find_all('tr')
    #Initialize a list to hold the extracted data
    data = []
   #Loop through each row, skipping the header row
    for row in rows[1:]:
        # Get all the columns in the current row
        cols = row.find_all('td')
        #Check if there are any columns in the row
        if cols:
            # Extract the bank name and market cap from the columns
            name = cols[1].text.strip()  # Get the bank name, removing extra whitespace
            # Remove unwanted characters from the Market Cap and convert it to float
            mc_usd = float(cols[2].text.strip().replace('\n', '').replace('$', '').replace(',', ''))
            # Append the extracted information as a tuple to the data list
            data.append((name, mc_usd))
    #Create a DataFrame from the extracted data
    df = pd.DataFrame(data, columns=table_attribs)
    # Return the DataFrame for further processing
    return df

In [ ]:
# Step 12: Define the URL of the webpage and the table attributes
url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"
table_attribs = ['Name', 'Market Cap (USD Billion)']  # Define the column names for the DataFrame

In [ ]:
#Call the extract function and store the returned DataFrame
df = extract(url, table_attribs)
print(df)

                                      Name  Market Cap (USD Billion)
0                           JPMorgan Chase                    432.92
1                          Bank of America                    231.52
2  Industrial and Commercial Bank of China                    194.56
3               Agricultural Bank of China                    160.68
4                                HDFC Bank                    157.91
5                              Wells Fargo                    155.87
6                        HSBC Holdings PLC                    148.90
7                           Morgan Stanley                    140.83
8                  China Construction Bank                    139.82
9                            Bank of China                    136.81


In [ ]:
#Log the completion of the data extraction
log_progress("Data extraction complete. Initiating Transformation process")

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving exchange_rate.csv to exchange_rate (1).csv


In [ ]:
#check uploaded files
import os
# List all files in the current directory
print(os.listdir())

['.config', 'code_log.txt', '.ipynb_checkpoints', 'exchange_rate.csv', 'sample_data']


In [ ]:
def transform(df, csv_path):
    ''' This function accesses the CSV file for exchange rate information and adds three columns to the data frame.'''

    # Read the exchange rates CSV into a DataFrame
    exchange_rates_df = pd.read_csv(csv_path)

    # Convert to dictionary: first column as keys, second column as values
    exchange_rate = pd.Series(exchange_rates_df['Rate'].values, index=exchange_rates_df['Currency']).to_dict()

    # Check if 'Market Cap (USD Billion)' exists, if so, rename it to 'MC_USD_Billion'
    if 'Market Cap (USD Billion)' in df.columns:
        df = df.rename(columns={'Market Cap (USD Billion)': 'MC_USD_Billion'})

    # Add transformed columns to the dataframe
    df['MC_GBP_Billion'] = [round(x * exchange_rate['GBP'], 2) for x in df['MC_USD_Billion']]
    df['MC_EUR_Billion'] = [round(x * exchange_rate['EUR'], 2) for x in df['MC_USD_Billion']]
    df['MC_INR_Billion'] = [round(x * exchange_rate['INR'], 2) for x in df['MC_USD_Billion']]

    return df

In [ ]:
transformed_df = transform(df, 'exchange_rate.csv')  # Replace with the actual filename you uploaded
print(transformed_df)

# Print the market cap of the 5th largest bank in billion EUR
print("Market Cap of 5th largest bank in EUR:", transformed_df['MC_EUR_Billion'][4])  # Adjusted to use transformed_df

                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   
5                              Wells Fargo          155.87          124.70   
6                        HSBC Holdings PLC          148.90          119.12   
7                           Morgan Stanley          140.83          112.66   
8                  China Construction Bank          139.82          111.86   
9                            Bank of China          136.81          109.45   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1          215.31        19204.58  
2          180.94        16138.75

In [ ]:
def load_to_csv(df, file_path):
    ''' This function saves the DataFrame to a specified CSV file. '''
    df.to_csv(file_path, index=False)  # Save the DataFrame to a CSV file without the index
    print(f"DataFrame saved to {file_path}")  # Log entry

In [ ]:
# Assuming 'transformed_df' is your DataFrame from the transformation step
load_to_csv(transformed_df, 'transformed_banks_data.csv')  # Adjust the filename as needed

DataFrame saved to transformed_banks_data.csv


In [ ]:
import os
# List files to verify the CSV file is created
print(os.listdir())

['.config', 'code_log.txt', '.ipynb_checkpoints', 'exchange_rate.csv', 'transformed_banks_data.csv', 'sample_data']


In [ ]:
def load_to_db(conn, table_name, df):
    ''' This function loads the DataFrame into the specified SQL table. '''
    df.to_sql(table_name, conn, if_exists='replace', index=False)  # Replace if the table already exists
    print(f"Data loaded into the table '{table_name}' in the database.")  # Log entry

In [ ]:
# Create a connection to the SQLite database
conn = sqlite3.connect('Banks.db')  # This will create the database if it doesn't exist

In [ ]:
# Assuming 'transformed_df' is your DataFrame from the transformation step
load_to_db(conn, 'Largest_banks', transformed_df)  # Load the data into the table

Data loaded into the table 'Largest_banks' in the database.


In [ ]:
# Close the database connection
conn.close()

In [ ]:
def run_queries(query, conn):
    ''' This function runs the provided SQL query and prints the results along with the query statement. '''
    cursor = conn.cursor()  # Create a cursor object to execute SQL commands
    cursor.execute(query)  # Execute the SQL query
    results = cursor.fetchall()  # Fetch all results

    # Print the query statement
    print(f"Query: {query}")
    # Print the results
    for row in results:
        print(row)  # Print each row of the results
    cursor.close()  # Close the cursor

In [ ]:
# Create a connection to the SQLite database
conn = sqlite3.connect('Banks.db')

In [ ]:
# Query to print the contents of the entire table
query1 = "SELECT * FROM Largest_banks"
run_queries(query1, conn)  # Execute and print results

# Query to print the average market capitalization of all the banks in Billion USD
query2 = "SELECT AVG(MC_GBP_Billion) FROM Largest_banks"
run_queries(query2, conn)  # Execute and print results

# Query to print only the names of the top 5 banks
query3 = "SELECT Name FROM Largest_banks LIMIT 5"
run_queries(query3, conn)  # Execute and print results

Query: SELECT * FROM Largest_banks
('JPMorgan Chase', 432.92, 346.34, 402.62, 35910.71)
('Bank of America', 231.52, 185.22, 215.31, 19204.58)
('Industrial and Commercial Bank of China', 194.56, 155.65, 180.94, 16138.75)
('Agricultural Bank of China', 160.68, 128.54, 149.43, 13328.41)
('HDFC Bank', 157.91, 126.33, 146.86, 13098.63)
('Wells Fargo', 155.87, 124.7, 144.96, 12929.42)
('HSBC Holdings PLC', 148.9, 119.12, 138.48, 12351.26)
('Morgan Stanley', 140.83, 112.66, 130.97, 11681.85)
('China Construction Bank', 139.82, 111.86, 130.03, 11598.07)
('Bank of China', 136.81, 109.45, 127.23, 11348.39)
Query: SELECT AVG(MC_GBP_Billion) FROM Largest_banks
(151.98700000000002,)
Query: SELECT Name FROM Largest_banks LIMIT 5
('JPMorgan Chase',)
('Bank of America',)
('Industrial and Commercial Bank of China',)
('Agricultural Bank of China',)
('HDFC Bank',)


In [ ]:
# Close the database connection
conn.close()

In [ ]:
rm code_log.txt

In [ ]:
# Define the logging function
def log_progress(message):
    """Log progress messages to code_log.txt."""
    with open('code_log.txt', 'a') as log_file:
        log_file.write(f"{message}\n")

# Define your data processing functions (extract, transform, load, etc.)
def extract():
    log_progress("Data extraction started.")
    # Add your data extraction logic here
    log_progress("Data extraction completed.")

def transform():
    log_progress("Data transformation started.")
    # Add your data transformation logic here
    log_progress("Data transformation completed.")

def load_to_csv():
    log_progress("Loading data to CSV started.")
    # Add your loading logic here
    log_progress("Data loaded to CSV successfully.")

def load_to_db():
    log_progress("Loading data to database started.")
    # Add your database loading logic here
    log_progress("Data loaded to database successfully.")

def main():
    log_progress("Starting the execution of the banks data processing project.")
    extract()
    transform()
    load_to_csv()
    load_to_db()
    log_progress("Final execution completed.")

# Run the main execution
main()

In [ ]:
# Read and print the contents of the log file
with open('code_log.txt', 'r') as log_file:
    log_contents = log_file.read()
    print(log_contents)

Starting the execution of the banks data processing project.
Data processing completed successfully.
Final execution completed.
Starting the execution of the banks data processing project.
Data extraction started.
Data extraction completed.
Data transformation started.
Data transformation completed.
Loading data to CSV started.
Data loaded to CSV successfully.
Loading data to database started.
Data loaded to database successfully.
Final execution completed.



In [ ]:
from google.colab import files
files.download('code_log.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>